In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

from bokeh.layouts import column
from bokeh.models import LinearAxis, Range1d
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output

import yfinance as yf
from utils import salib as sa
import numpy as np
import pandas as pd
#from datetime import datetime, timedelta

In [8]:
# Load financial data into a pandas DataFrame
ticker = 'AAPL'
start_date = '2000-01-01'
end_date = '2024-01-01'
data = yf.download(ticker, start=start_date, end=end_date)
N = len(data)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,0.936384,1.004464,0.907924,0.999442,0.844004,535796800
2000-01-04,0.966518,0.987723,0.903460,0.915179,0.772846,512377600
2000-01-05,0.926339,0.987165,0.919643,0.928571,0.784155,778321600
2000-01-06,0.947545,0.955357,0.848214,0.848214,0.716296,767972800
2000-01-07,0.861607,0.901786,0.852679,0.888393,0.750226,460734400
...,...,...,...,...,...,...
2023-12-22,195.179993,195.410004,192.970001,193.600006,192.868134,37122800
2023-12-26,193.610001,193.889999,192.830002,193.050003,192.320221,28919300
2023-12-27,192.490005,193.500000,191.089996,193.149994,192.419830,48087700


# Entropy

## Shannon Entropy

In [11]:
length = 100

df = pd.DataFrame(index=data.index, columns=["indicator"])
print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1, N):
    
    x = data.Close[t - (length - 1):t + 1].values
    se = sa.Entropy.shannon(x) # Scipy method
    #se = shannon_entropy(x) # My method
    df.loc[df.index[t]] = [se]
        
    print("t = %d; Shannon entropy = %f" % (t, se), end='')
    print("\t\t\t", end='\r')

Candle range = 100; Data length = 6037



## Approximate Entropy

In [14]:
length = 100

df = pd.DataFrame(index=data.index, columns=["indicator"])
print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1, N):
    
    x = data.Close[t - (length - 1):t + 1].values
    ae = sa.Entropy.approximate(x, m=2, r=3)        
    df.loc[df.index[t]] = [ae]
        
    print("t = %d; Approximate entropy = %f" % (t, ae), end='')
    print("\t\t\t", end='\r')

Candle range = 100; Data length = 6037



# Chaos

## Lyapunov Exponent

In [ ]:
length = 48

df = pd.DataFrame(index=dataframe.index, columns=["indicator"])
print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1, N):
    
    x = dataframe.close[t - (length - 1):t + 1].values
    
    # Diameter selection
    diameter_set = 2**np.linspace(0, -50, 200) * 5
    
    for diameter in diameter_set:
        counter = 0
        for i in range(length):
            for j in range(i + 1, length):

                if np.abs(x[i] - x[j]) <= diameter:
                    counter += 1
        if counter <= 15: break

    # Lyapunov exponent
    lyapunov = lyapunov_exponent(x, candle_range=length, initial_diameter=diameter, display=False)
    df.loc[df.index[t]] = [lyapunov]
        
    print("t = %d; Lyapunov exp. = %f" % (t, lyapunov), end='')
    print("\t\t\t", end='\r')

# Mutual Information

## Constant delay

In [ ]:
length = 100
tau = 7 # Delay time (should be less than 'length')

df = pd.DataFrame(index=dataframe.index, columns=["indicator"])
print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1 + tau, N):
    
    unlagged = dataframe.close[t - (length - 1) - tau:t + 1 - tau].values
    lagged = dataframe.close[t - (length - 1):t + 1].values
    
    info = mutual_info(unlagged, lagged, local=False)
    df.loc[df.index[t]] = [info]
        
    print("t = %d; Mutual info. = %f" % (t, info), end='')
    print("\t\t\t", end='\r')

## First minimum

In [ ]:
length = 25
tau_max = 20 #int(length / 2)

#=============================== CALCULATIONS ================================#
df = pd.DataFrame(index=dataframe.index, columns=["indicator"]) # empty dataframe for results

print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1, N):
    
    x = dataframe.close[t - (length - 1):t + 1].values
        
    information = []
    for tau in range(1, tau_max + 1):
    
        #------------------------ TIME SERIES SECTION ------------------------#
        unlagged = x[:-tau]
        lagged = np.roll(x, -tau)[:-tau]
        
        if len(unlagged):

            info = mutual_info(unlagged, lagged, local=False)
            information.append(info)
            
            print("t = %d; tau = %d; mutual information = %.5f" % (t, tau, info), end='')
            print("\t\t\t", end='\r')

            if len(information) > 1 and information[-2] < information[-1]: # first local minimum
                first_minimum = tau - 1
                df.loc[df.index[t]] = [first_minimum]
                break
            #else:
            #    first_minimum = min(information)
            #    df.loc[df.index[t]] = [first_minimum]

# Complexity

## Lempel-Ziv Complexity

In [ ]:
length = 100

#=============================== CALCULATIONS ================================#
df = pd.DataFrame(index=dataframe.index, columns=["indicator"]) # empty dataframe for results

print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1, N):
    
    #-------------------------- TIME SERIES SECTION --------------------------#
    x = dataframe.close[t - (length - 1):t + 1].values

    #---------------------------- SHANNON ENTROPY ----------------------------#
    binary_sequence = binarizer(x)
    complexity = lempel_ziv_complexity(np.array(binary_sequence))
    df.loc[df.index[t]] = [complexity]
        
    print("t = %d; Lempel-Ziv complexity = %f" % (t, complexity), end='')
    print("\t\t\t", end='\r')

# Multifractal Analysis

## Overview

In [ ]:
length = 2**7 # choose length as a power of 2

n = length + 1000
ts = dataframe.close[n - length:n]

In [ ]:
#==================== PREPROCESS ====================#
s = (ts - np.mean(ts)) / np.std(ts) # standardization
s = 1 / (1 + np.exp(-np.array(s))) # pass through sigmoid function

#==================== PLOT DATA ====================#
fig = plt.figure(constrained_layout=True, figsize=(11, 7))    
plt.style.use('classic')
fig.set_facecolor('none')
linewidths, fontsizes = 2.5, 15
plt.rc('lines', linewidth=linewidths)
plt.rc('axes', linewidth=linewidths)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams.update({'mathtext.default':'regular'})

gs = fig.add_gridspec(2, 1)
ax1 = fig.add_subplot(gs[0,0:])
ax1.set_title(filename, fontsize=fontsizes+5, pad=15)
ax1.plot(ts, linestyle='-', marker='', markersize=10, color='darkblue')
ax1.tick_params(axis="both", direction="in", left=True, width=linewidths, length=4, labelsize=fontsizes)
ax1.set_xlabel(r"$Date$", fontsize=fontsizes+5)
ax1.set_ylabel(PAIR_NAME, fontsize=fontsizes+5)
ax1 = fig.add_subplot(gs[1,0:])
ax1.set_title("DATA AFTER PREPROCESS", fontsize=fontsizes+5, pad=15)
ax1.plot(s, linestyle='-', marker='', markersize=10, color='lightblue')
ax1.tick_params(axis="both", direction="in", left=True, width=linewidths, length=4, labelsize=fontsizes)
ax1.set_xlabel(r"$Data \ Point \ Number$", fontsize=fontsizes+5)
ax1.set_ylabel(r"$Data \ [arb. unit]$", fontsize=fontsizes+5)
plt.show()

In [ ]:
#----------------------- PARAMETERS -----------------------#
l = floor(np.log2(len(s)))
L = 2**l
q_range = (-40, 40)
scale_range = (1, l)
q_values = np.arange(q_range[0], q_range[1] + 1)
scale_values = np.arange(scale_range[0], scale_range[1] + 1)

#---------------- CHHABRA-JENSEN CALCULATION ----------------#
results = chhabra_jensen_analysis(s[:L], q_values, scale_values) # output: [alpha, falpha, Dq, Rsquared_alpha, Rsquared_falpha, Rsquared_Dq, log_l, Ma, Mf, Md]
alpha = results[0]
falpha = results[1]

#------------------------- not-NaNs -------------------------#
alpha_notnans = ~ np.isnan(alpha)
falpha_notnans = ~ np.isnan(falpha)
notnan_indices = alpha_notnans & falpha_notnans
alpha = alpha[notnan_indices]
falpha = falpha[notnan_indices]


#======================= PLOT =======================#
fig = plt.figure(constrained_layout=True, figsize=(14, 8))
plt.style.use('classic')
fig.set_facecolor('none')
linewidths, fontsizes = 2.5, 15
plt.rc('lines', linewidth=linewidths)
plt.rc('axes', linewidth=linewidths)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams.update({'mathtext.default':'regular'})
gs = fig.add_gridspec(2, 2)

#--------------------- LEFT PANEL ---------------------#
ax1 = fig.add_subplot(gs[0:,0])
ax1.set_title("MULTIFRACTAL SPECTRUM\n" + filename, fontsize=fontsizes+5, pad=15)
ax1.plot(results[0], results[1], linestyle='-', marker='.', markersize=10, color='k')
ax1.tick_params(axis="both", direction="in", left=True, width=linewidths, length=4, labelsize=fontsizes)
ax1.set_xlabel(r"$Hölder \ exponent \ \alpha$", fontsize=fontsizes+5)
ax1.set_ylabel(r"$Hausdorff \ dimension \ f(\alpha)$", fontsize=fontsizes+5)
#ax1.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
#ax1.set_xlabel(r"$\alpha$", fontsize=fontsizes)
amin_index = np.where(results[0]==min(results[0]))[0]
amax_index = np.where(results[0]==max(results[0]))[0]
ax1.text(0.5, 0.50, r"$\Delta\alpha=%.3f$" % (max(results[0]) - min(results[0])), color='k', fontsize=fontsizes, ha='center', va='center', transform=ax1.transAxes)
ax1.text(0.5, 0.450, r"$f(\alpha_{max})-f(\alpha_{min})=%.3f$" % abs(results[1][amax_index] - results[1][amin_index]), color='k', fontsize=fontsizes, ha='center', va='center', transform=ax1.transAxes)
#ax1.text(0.1, 0.80, "$m_1=7$ \n$m_2=8$ \n \n$p=4$ \n$p_c=4$", fontsize=fontsizes, ha='center', va='center', transform=ax.transAxes) 
ax1.axis([min(results[0]) - 0.1, max(results[0]) + 0.1, min(results[1]) - 0.1, max(results[1]) + 0.1])

#------------------- UPPER LEFT PANEL -------------------#
ax2 = fig.add_subplot(gs[0,1])
ax2.set_title("$Scaling \ of \ the \ partition \ function \ Z(l)$", fontsize=fontsizes+5, pad=15)
ax2.set_xlabel("$log \ of \ the \ lentgh \ scales, \ log_{10}(l)$", fontsize=fontsizes+5)
ax2.set_ylabel("$log \ of \ the \ q-th \ moment, \ log_{10}(Z_q(l))$", fontsize=fontsizes+5)
ax2.set_xlim([min(results[-4]) - .3, max(results[-4]) + .3])
ax2.set_ylim([int(min(results[-1][-1])) - 10, int(max(results[-1][0])) + 10])
color_palette = plt.cm.get_cmap('viridis', len(results[-1]))
for i in range(0, len(results[-1]), int(q_range[1]/5)):
    ax2.plot(results[-4], results[-1][i], marker='o', ms=10, ls='-', color=color_palette(i), label=f"$q={q_values[i]}$")
ax2.legend(ncol=1, prop={'size':12}, bbox_to_anchor=(1.23, 1.05), numpoints=1,
           labelspacing=0.4, handlelength=2, handletextpad=0.8, framealpha=0)

#------------------- DOWN RIGHT PANEL -------------------#
ax3 = fig.add_subplot(gs[1,1])
ax3.set_title("$Generalized \ fractal \ dimension \ spectrum$", fontsize=fontsizes+5, pad=15)
ax3.plot(q_values, results[2], marker='o', ms=10, ls='', color='k')
#ax3.axis([q_range[0]-1, q_range[-1]+1, Dq[0]-1, Dq[-1]+1])
ax3.set_xlabel("$Moment \ order \ q$", fontsize=fontsizes+5)
ax3.set_ylabel("$Fractal \ dimension \ D(q)$", fontsize=fontsizes+5)

plt.show()

## Indicator-1: Multifractal Spectrum Width

In [ ]:
length = 2**7
q_range = (-40, 40)

warnings.filterwarnings(action="ignore")

#=============================== CALCULATIONS ================================#
df = pd.DataFrame(index=dataframe.index, columns=["indicator"]) # empty dataframe for results

print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1, N):
    
    #-------------------------- TIME SERIES SECTION --------------------------#
    x = dataframe.close[t - (length - 1):t + 1].values

    
    #========================= MULTIFRACTAL ANALYSIS =========================#
    
    #------------------------------- PREPROCESS ------------------------------#
    #x = (x - np.mean(x)) / np.std(x) # standardization
    #x = 1 / (1 + np.exp(-np.array(x))) # passing through sigmoid function
    
    #----------------------- CHHABRA-JENSEN CALCULATION ----------------------#
    l = floor(np.log2(len(x)))
    scale_range = (1, l)
    q_values = np.arange(q_range[0], q_range[1] + 1)
    scale_values = np.arange(scale_range[0], scale_range[1] + 1)
    
    results = chhabra_jensen_analysis(x, q_values, scale_values) # output: [alpha, falpha, Dq, Rsquared_alpha, Rsquared_falpha, Rsquared_Dq, log_l, Ma, Mf, Md]
    delta_alpha = max(results[0]) - min(results[0])
    df.loc[df.index[t]] = [*delta_alpha]
        
    print("t = %d; Multifractal spectrum width = %f" % (t, delta_alpha), end='')
    print("\t\t\t", end='\r')

## Indicator-2: Multifractal Spectrum Height

In [ ]:
length = 2**7
q_range = (-40, 40)

warnings.filterwarnings(action="ignore")

#=============================== CALCULATIONS ================================#
df = pd.DataFrame(index=dataframe.index, columns=["indicator"]) # empty dataframe for results

print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1, N):
    
    #-------------------------- TIME SERIES SECTION --------------------------#
    x = dataframe.close[t - (length - 1):t + 1].values

    
    #========================= MULTIFRACTAL ANALYSIS =========================#
    
    #------------------------------- PREPROCESS ------------------------------#
    #x = (x - np.mean(x)) / np.std(x) # standardization
    #x = 1 / (1 + np.exp(-np.array(x))) # passing through sigmoid function
    
    #----------------------- CHHABRA-JENSEN CALCULATION ----------------------#
    l = floor(np.log2(len(x)))
    scale_range = (1, l)
    q_values = np.arange(q_range[0], q_range[1] + 1)
    scale_values = np.arange(scale_range[0], scale_range[1] + 1)
    
    results = chhabra_jensen_analysis(x, q_values, scale_values) # output: [alpha, falpha, Dq, Rsquared_alpha, Rsquared_falpha, Rsquared_Dq, log_l, Ma, Mf, Md]
    
    amin_index = np.where(results[0]==min(results[0]))[0]
    amax_index = np.where(results[0]==max(results[0]))[0]
    delta_falpha = abs(results[1][amax_index] - results[1][amin_index])
    df.loc[df.index[t]] = [*delta_falpha]
        
    print("t = %d; Multifractal spectrum height = %f" % (t, delta_falpha), end='')
    print("\t\t\t", end='\r')

# Recurrence Quantification Analysis

In [ ]:
from pyrqa.time_series import TimeSeries
from pyrqa.settings import Settings
from pyrqa.analysis_type import Classic
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation, RPComputation
from pyrqa.image_generator import ImageGenerator

## Indicator-1: Recurrence Rate

In [ ]:
length = 100

#=============================== CALCULATIONS ================================#
df = pd.DataFrame(index=dataframe.index, columns=["indicator"]) # empty dataframe for results

print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1, N):
    
    #-------------------------- TIME SERIES SECTION --------------------------#
    x = dataframe.close[t - (length - 1):t + 1].values
    
    #----------------------------- RQA ANALYSIS ------------------------------#
    reconstructed = TimeSeries(x, embedding_dimension=2, time_delay=2)
    settings = Settings(reconstructed,  analysis_type=Classic,  neighbourhood=FixedRadius(0.65), similarity_measure=EuclideanMetric,  theiler_corrector=1)
    computation = RQAComputation.create(settings, verbose=False)
    result = computation.run()
    RR = result.recurrence_rate
    df.loc[df.index[t]] = [RR]
        
    print("t = %d; RR = %f" % (t, RR), end='')
    print("\t\t\t", end='\r')

## Indicator-2: Determinism

In [ ]:
length = 100

#=============================== CALCULATIONS ================================#
df = pd.DataFrame(index=dataframe.index, columns=["indicator"]) # empty dataframe for results

print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1, N):
    
    #-------------------------- TIME SERIES SECTION --------------------------#
    x = dataframe.close[t - (length - 1):t + 1].values
    
    #----------------------------- RQA ANALYSIS ------------------------------#
    reconstructed = TimeSeries(x, embedding_dimension=2, time_delay=2)
    settings = Settings(reconstructed,  analysis_type=Classic,  neighbourhood=FixedRadius(0.65), similarity_measure=EuclideanMetric,  theiler_corrector=1)
    computation = RQAComputation.create(settings, verbose=False)
    result = computation.run()
    DET = result.determinism
    df.loc[df.index[t]] = [DET]
        
    print("t = %d; DET = %f" % (t, DET), end='')
    print("\t\t\t", end='\r')

## Indicator-3: Laminarity

In [ ]:
length = 100

#=============================== CALCULATIONS ================================#
df = pd.DataFrame(index=dataframe.index, columns=["indicator"]) # empty dataframe for results

print("Candle range = %d; Data length = %d" % (length, N), end="\n\n")

for t in range(length - 1, N):
    
    #-------------------------- TIME SERIES SECTION --------------------------#
    x = dataframe.close[t - (length - 1):t + 1].values
    
    #----------------------------- RQA ANALYSIS ------------------------------#
    reconstructed = TimeSeries(x, embedding_dimension=2, time_delay=2)
    settings = Settings(reconstructed,  analysis_type=Classic,  neighbourhood=FixedRadius(0.65), similarity_measure=EuclideanMetric,  theiler_corrector=1)
    computation = RQAComputation.create(settings, verbose=False)
    result = computation.run()
    LAM = result.laminarity

    df.loc[df.index[t]] = [LAM]
        
    print("t = %d; LAM = %f" % (t, LAM), end='')
    print("\t\t\t", end='\r')

## Recurrence Plot

In [ ]:
from __future__ import division, print_function
import pylab
from scipy.spatial.distance import pdist, squareform


def recurrence_plot(s, eps=0.10, steps=10):
    
    d = pdist(s[:, None])
    d = np.floor(d / eps)
    d[d>steps] = steps
    Z = squareform(d)
    return Z


def moving_average(s, r=5):
    
    return np.convolve(s, np.ones((r,))/r, mode='valid')


if __name__ == "__main__":
    
    # generate singal
    N = 200
    eps = 0.1
    steps = 10

    # plot unifrom dist filtered with moving average
    ru = np.random.uniform(low=-1, high=1, size=N)
    ru_filtered = moving_average(ru)

    pylab.title("Normal")
    pylab.subplot(221)
    pylab.plot(ru_filtered)
    pylab.title("Unitary")
    pylab.subplot(223)
    pylab.imshow(recurrence_plot(ru_filtered, eps=eps, steps=steps))

    # plot normal dist filtered with moving average
    rn = np.random.normal(size=N)
    rn_filtered = moving_average(rn)

    pylab.subplot(222)
    pylab.plot(rn_filtered)
    pylab.title("Normal")
    pylab.subplot(224)
    pylab.imshow(recurrence_plot(rn_filtered, eps=eps, steps=steps))

    pylab.show()

In [ ]:
eps = 0.3
steps = 10

# calculate two different data section
x = dataframe.close["2021-01-01":"2022-01-01"]
recurrence_x = recurrence_plot(x.values, eps=eps, steps=steps)

y = dataframe.close["2022-01-01":"2023-01-01"]
recurrence_y = recurrence_plot(y.values, eps=eps, steps=steps)

#================================= PLOT DATA =================================#
fig = plt.figure(constrained_layout=True, figsize=(12, 7))  
plt.style.use('classic')
fig.set_facecolor('white')
plt.rcParams['font.family'] = 'Arial'
plt.rcParams.update({'mathtext.default':'regular'})

gs = fig.add_gridspec(4, 2)

ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(x, linestyle='-', color="k")
ax1.set_xlabel(r"$Date$", fontsize=10)
ax1.set_ylabel(PAIR_NAME, fontsize=10)

ax2 = fig.add_subplot(gs[1:,0])
ax2.imshow(recurrence_x)


ax3 = fig.add_subplot(gs[0, 1])
ax3.plot(y, linestyle='-', color="k")
ax3.set_xlabel(r"$Date$", fontsize=10)
ax3.set_ylabel(PAIR_NAME, fontsize=10)

ax4 = fig.add_subplot(gs[1:,1])
ax4.imshow(recurrence_y)
plt.show()

# Visiualize

## Interactive Chart: Bokeh

In [ ]:
candle_body_width = (timedelta(minutes=TIME_SCALE)).total_seconds() * 1000 / 2 # half of the trading time in miliseconds
fig_width = 1250

tools = "pan, reset, wheel_zoom"

#=============================== CANDLE CHART ================================#
candChart = figure(x_axis_type="datetime", tools=tools, width=fig_width, height=400, title="Price Chart")
# green bars
candChart.segment(x0=dataframe.index[inc], y0=dataframe.high[inc], x1=dataframe.index[inc], y1=dataframe.low[inc], color="green")
candChart.vbar(x=dataframe.index[inc], width=candle_body_width, bottom=dataframe.open[inc], top=dataframe.close[inc], fill_color="green", line_color="green")
# red bars
candChart.segment(x0=dataframe.index[dec], y0=dataframe.high[dec], x1=dataframe.index[dec], y1=dataframe.low[dec], color="red")
candChart.vbar(x=dataframe.index[dec], width=candle_body_width, top=dataframe.open[dec], bottom=dataframe.close[dec], fill_color="red", line_color="red")
candChart.xaxis.axis_label="Date"
candChart.yaxis.axis_label=PAIR

#=============================== VOLUME CHART ================================#
#volChart = figure(x_axis_type="datetime", width=fig_width, height=200)
#volChart.vbar(dataframe.index[inc], width=width, top=dataframe.Volume[inc], fill_color="green", line_color="green", alpha=0.8)
#volChart.vbar(dataframe.index[dec], width=width, top=dataframe.Volume[dec], fill_color="red", line_color="red", alpha=0.8)
#volChart.xaxis.axis_label="Date"
#volChart.yaxis.axis_label="Volume"

#============================== INDICATOR CHART ==============================#
indicatorChart = figure(x_axis_type="datetime", tools=tools, x_range=candChart.x_range, width=fig_width, height=300, title="Indicator")
indicatorChart.line(df.index, df.indicator, line_color="purple", line_width=1.5, name="Shannon Entropy")
indicatorChart.xaxis.axis_label="Date"
indicatorChart.yaxis.axis_label = INDICATOR

col = column(candChart, indicatorChart)
show(col)

## Static Plot: Matplotlib

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
plt.style.use('classic')
fig.set_facecolor('white')
linewidths, fontsizes = 2., 15
plt.rc('lines', linewidth=linewidths)
plt.rc('axes', linewidth=linewidths)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams.update({'mathtext.default':'regular'})

ax.plot(dataframe.close, ls='-',color='blue', lw=.7)
#ax.set_xlim([0,1400])
ax.tick_params(axis="x", direction="in", labelcolor='black', width=linewidths, length=4)
ax.tick_params(axis="y", direction="in", labelcolor='blue', width=linewidths, length=4)
ax.set_xlabel(r"$Date$", fontsize=fontsizes)
ax.set_ylabel(PAIR, fontsize=fontsizes)

ax_indicator = ax.twinx()
ax_indicator.plot(df.indicator, color='green', lw=.8)
#ax_indicator.set_xlim([1380,1390])
#ax_indicator.set_ylim([-1, 2])
ax_indicator.tick_params(axis="both", direction="in", left=True, labelcolor='green', width=linewidths, length=4)
ax_indicator.set_ylabel(INDICATOR, fontsize=fontsizes)
plt.show()